In [23]:
# ============================
# Project Two: Grazioso Salvare Dashboard
# ============================

# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash
JupyterDash.infer_jupyter_proxy_config()

# Dash / Components
from dash import dcc, html, dash_table, Input, Output
import dash_leaflet as dl
import plotly.express as px

# Data / Utils
import pandas as pd
import base64, shutil, os
from pathlib import Path


from CRUD_Python_Module import AnimalShelter


# -----------------------------
# CONFIG (edit as needed)
# -----------------------------
UNIQUE_ID = "RM-CS340"        
MONGO_USER = "aacuser"
MONGO_PASS = "SNHU1234"
DB_NAME    = "aac"           
COLL_NAME  = "animals"


LOGO_PATH = "./Grazioso Salvare Logo.png"


# -----------------------------
# Model / CRUD setup
# -----------------------------
shelter = AnimalShelter(
    user=MONGO_USER,
    password=MONGO_PASS,
    host="localhost",
    port=27017,
    db=DB_NAME,
    col=COLL_NAME
)

def load_all():
    """Load all documents into a DataFrame (unfiltered)."""
    recs = shelter.read({})
    df = pd.DataFrame(recs)
    df.drop(columns=['_id'], inplace=True, errors='ignore')
    return df



def encode_image(path_str):
    p = Path(path_str)
    if not p.exists():
        return None
    b64 = base64.b64encode(p.read_bytes()).decode("utf-8")
    return f"data:image/png;base64,{b64}"

LOGO_SRC = encode_image(LOGO_PATH)



RESCUE_SPECS = {
    "Water Rescue": {
        "breeds": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"],
        "sex": "Intact Female",
        "age_weeks": (26, 156),
    },
    "Mountain or Wilderness Rescue": {
        "breeds": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"],
        "sex": "Intact Male",
        "age_weeks": (26, 156),
    },
    "Disaster or Individual Tracking": {
        "breeds": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"],
        "sex": "Intact Male",
        "age_weeks": (20, 300),
    },
}

def build_query(selection: str):
    """Return Mongo query for the selected rescue type, or {} for Reset."""
    if selection == "Reset" or selection is None:
        return {}
    spec = RESCUE_SPECS[selection]
    min_w, max_w = spec["age_weeks"]
    return {
        "$and": [
            {"animal_type": "Dog"},
            {"breed": {"$in": spec["breeds"]}},
            {"sex_upon_outcome": spec["sex"]},
            {"age_upon_outcome_in_weeks": {"$gte": min_w, "$lte": max_w}},
        ]
    }


# -----------------------------
# Initial data
# -----------------------------
df_all = load_all()


# -----------------------------
# App layout
# -----------------------------
app = JupyterDash(__name__)

app.layout = html.Div(
    [
        # Header: Logo + Title + Unique ID
        html.Div(
            [
                html.A(
                    html.Img(src=LOGO_SRC, style={"height": "80px"}) if LOGO_SRC else html.Div("Grazioso Salvare"),
                    href="", target="_blank", style={"textDecoration": "none"}
                ),
                html.Div(
                    [
                        html.H2("Grazioso Salvare — Search & Rescue Dashboard", style={"margin": "6px 0 0 0"}),
                        html.Div(f"Unique ID: {UNIQUE_ID}", style={"fontWeight": "bold", "marginTop": "4px"}),
                    ]
                ),
            ],
            style={"display": "flex", "gap": "16px", "alignItems": "center", "marginBottom": "12px"},
        ),

        html.Hr(),

        # FILTERS
        html.Div(
            [
                html.Label("Rescue Filter"),
                dcc.RadioItems(
                    id="rescue-filter",
                    options=[
                        {"label": "Water Rescue", "value": "Water Rescue"},
                        {"label": "Mountain or Wilderness Rescue", "value": "Mountain or Wilderness Rescue"},
                        {"label": "Disaster or Individual Tracking", "value": "Disaster or Individual Tracking"},
                        {"label": "Reset (All)", "value": "Reset"},
                    ],
                    value="Reset",
                    inputStyle={"marginRight": "6px", "marginLeft": "14px"},
                    labelStyle={"display": "inline-block", "marginRight": "14px"},
                ),
            ],
            style={"marginBottom": "10px"},
        ),

        # DATA TABLE
        dash_table.DataTable(
            id="datatable-id",
            columns=[{"name": c, "id": c, "deletable": False, "selectable": True} for c in df_all.columns],
            data=df_all.to_dict("records"),
            row_selectable="single",
            selected_rows=[0],  
            page_size=10,
            sort_action="native",
            filter_action="native",
            style_table={"overflowX": "auto", "maxHeight": "500px", "overflowY": "auto"},
            style_cell={"textAlign": "left", "minWidth": "120px", "whiteSpace": "normal", "height": "auto"},
            style_header={"fontWeight": "bold"},
        ),

        html.Br(),
        html.Hr(),

        
        html.Div(
            [
                html.Div(id="map-id", className="col s12 m6"),
                html.Div(
                    [
                        html.H4("Breed Distribution"),
                        dcc.Graph(id="pie-id", figure={}),
                    ],
                    style={"minWidth": "420px", "flex": "1"},
                ),
            ],
            style={"display": "flex", "gap": "24px", "flexWrap": "wrap"},
        ),
    ],
    style={"padding": "12px", "fontFamily": "Arial, sans-serif"},
)


# -----------------------------
# Controller / Callbacks
# -----------------------------

# Highlight selected columns 
@app.callback(
    Output("datatable-id", "style_data_conditional"),
    Input("datatable-id", "selected_columns"),
)
def _highlight_cols(selected_columns):
    selected_columns = selected_columns or []
    return [{"if": {"column_id": c}, "background_color": "#D2F3FF"} for c in selected_columns]


# 1) FILTER → TABLE data
@app.callback(
    Output("datatable-id", "data"),
    Input("rescue-filter", "value"),
)
def _apply_filter(selection):
    query = build_query(selection)
    if query == {}:
        df = load_all()
    else:
        recs = shelter.read(query)
        df = pd.DataFrame(recs)
        df.drop(columns=["_id"], inplace=True, errors="ignore")
    return df.to_dict("records") if not df.empty else []


# 2) TABLE (data/selected_rows) → MAP
@app.callback(
    Output("map-id", "children"),
    Input("datatable-id", "data"),
    Input("datatable-id", "selected_rows"),
)
def _update_map(table_data, selected_rows):
    if not table_data:
        return [html.Div("No data to map.")]
    dff = pd.DataFrame(table_data)
    if dff.empty:
        return [html.Div("No data to map.")]

    # normalize selection
    row = 0 if not selected_rows else max(0, min(selected_rows[0], len(dff) - 1))

    # Prefer named columns from AAC dataset
    lat_col = next((c for c in ["location_lat", "latitude", "lat", "Y", "Location_Latitude"] if c in dff.columns), None)
    lon_col = next((c for c in ["location_long", "longitude", "lon", "X", "Location_Longitude"] if c in dff.columns), None)
    breed_col = next((c for c in ["breed", "Breed"] if c in dff.columns), None)
    name_col  = next((c for c in ["name", "Name"] if c in dff.columns), None)

    if not (lat_col and lon_col):
        return [html.Div("Latitude/Longitude columns not found in data.")]

    lat = pd.to_numeric(dff.loc[row, lat_col], errors="coerce")
    lon = pd.to_numeric(dff.loc[row, lon_col], errors="coerce")
    if pd.isna(lat) or pd.isna(lon):
        return [html.Div("Selected row has no valid coordinates.")]

    tooltip_txt = str(dff.loc[row, breed_col]) if breed_col else "Breed"
    popup_name  = str(dff.loc[row, name_col])  if name_col  else "Unknown"

    return [
        dl.Map(
            style={"width": "1000px", "height": "500px"},
            center=[30.75, -97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(
                    position=[float(lat), float(lon)],
                    children=[
                        dl.Tooltip(tooltip_txt),
                        dl.Popup([html.H1("Animal Name"), html.P(popup_name)]),
                    ],
                ),
            ],
        )
    ]


# 3) TABLE (data) → PIE chart (breeds)
@app.callback(
    Output("pie-id", "figure"),
    Input("datatable-id", "data"),
)
def _update_pie(table_data):
    if not table_data:
        return {}
    dff = pd.DataFrame(table_data)
    if "animal_type" in dff.columns:
        dff = dff[dff["animal_type"] == "Dog"]
    if "breed" in dff.columns and not dff["breed"].dropna().empty:
        dff = dff[dff["breed"].notna()]
        fig = px.pie(dff, names="breed", title="Breeds in Selection")
        return fig
    if "outcome_type" in dff.columns:
        fig = px.pie(dff, names="outcome_type", title="Outcome Types in Selection")
        return fig
    return {}



app.run_server(mode="inline", port=8051, debug=False)


 * Running on http://127.0.0.1:8051/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Oct/2025 03:48:01] "GET /_alive_ade6d4d7-6904-479d-b5e8-2adccb237f11 HTTP/1.1" 200 -


127.0.0.1 - - [19/Oct/2025 03:48:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2025 03:48:02] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2025 03:48:02] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2025 03:48:03] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 304 -
127.0.0.1 - - [19/Oct/2025 03:48:03] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 304 -
127.0.0.1 - - [19/Oct/2025 03:48:03] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [19/Oct/2025 03:48:03] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [19/Oct/2025 03:48:03] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2025 03:48:04] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2025 03:48:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2025 03:48:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - -